In [1]:
import pandas as pd
import numpy as np
import os
import json   
import statsmodels.api as sm

In [20]:
base_dir = '/Users/siddharthadatta/Library/CloudStorage/OneDrive-Personal/MFE tasks/Capstone project/Project_code/WQU_Capstone_Project/'
index_cons_with_risk_metrics_file = 'nifty50_cons_with_risk_metrics.csv'
index_values_file = 'nifty50_index_prices_2015_2025.csv'

In [21]:
price_data = pd.read_csv(base_dir + 'all_price_data.csv', parse_dates=['Date'])
price_data['daily_return'] = price_data.groupby('Symbol')['Close'].pct_change()
price_data.dropna(inplace=True)
price_data

,Symbol,Date,Close,Volume,daily_return
1,BHARTIARTL,2015-01-02,309.898743,2795887,0.005924
2,BHARTIARTL,2015-01-05,303.022461,1605267,-0.022189
3,BHARTIARTL,2015-01-06,300.645477,3916948,-0.007844
4,BHARTIARTL,2015-01-07,301.367035,4899318,0.002400
5,BHARTIARTL,2015-01-08,306.587952,7096620,0.017324
...,...,...,...,...,...
188365,M&M,2025-12-23,3625.100098,1413628,0.002323
188366,M&M,2025-12-24,3636.699951,842897,0.003200
188367,M&M,2025-12-26,3623.100098,829910,-0.003740
188368,M&M,2025-12-29,3592.100098,1034692,-0.008556


In [22]:
index_values = pd.read_csv(os.path.join(base_dir, index_values_file))
index_values

,Date,Adj Close,Close,High,Low,Open,Volume
0,2015-01-02,8395.450195,8395.450195,8410.599609,8288.700195,8288.700195,101900
1,2015-01-05,8378.400391,8378.400391,8445.599609,8363.900391,8407.950195,118200
2,2015-01-06,8127.350098,8127.350098,8327.849609,8111.350098,8325.299805,172800
3,2015-01-07,8102.100098,8102.100098,8151.200195,8065.450195,8118.649902,164100
4,2015-01-08,8234.599609,8234.599609,8243.500000,8167.299805,8191.399902,143800
...,...,...,...,...,...,...,...
2702,2025-12-23,26177.150391,26177.150391,26233.550781,26119.050781,26205.199219,216600
2703,2025-12-24,26142.099609,26142.099609,26236.400391,26123.000000,26170.650391,188800
2704,2025-12-26,26042.300781,26042.300781,26144.199219,26008.599609,26121.250000,142200
2705,2025-12-29,25942.099609,25942.099609,26106.800781,25920.300781,26063.349609,234300


In [23]:
index_cons_with_risk_metrics = pd.read_csv(os.path.join(base_dir, index_cons_with_risk_metrics_file))
index_cons_with_risk_metrics

,Date,Index_name,Security Symbol,ff_mcap,Beta,Volatility
0,2016-03-31,Nifty 50,ACC,13003.10,0.840607,0.238163
1,2016-03-31,Nifty 50,INFY,243457.99,0.818393,0.274475
2,2016-03-31,Nifty 50,KOTAKBANK,69877.08,1.094524,0.274023
3,2016-03-31,Nifty 50,LT,99713.71,1.193147,0.285038
4,2016-03-31,Nifty 50,LUPIN,35321.94,0.734866,0.344207
...,...,...,...,...,...,...
933,2025-09-30,Nifty 50,HINDALCO,NaN,1.216176,0.277803
934,2025-09-30,Nifty 50,HINDUNILVR,NaN,0.441855,0.195020
935,2025-09-30,Nifty 50,ICICIBANK,NaN,0.880423,0.173273
936,2025-09-30,Nifty 50,COALINDIA,NaN,0.933043,0.213540


Add Signal to noise ratio

In [24]:
index_narrowing_1 = index_cons_with_risk_metrics.copy()

In [25]:
index_narrowing_1['SNR'] = index_narrowing_1['Beta'] / index_narrowing_1['Volatility']**2
index_narrowing_1['rank_snr'] = index_narrowing_1['SNR'].groupby(index_narrowing_1['Date']).rank(ascending=False)
index_narrowing_1

,Date,Index_name,Security Symbol,ff_mcap,Beta,Volatility,SNR,rank_snr
0,2016-03-31,Nifty 50,ACC,13003.10,0.840607,0.238163,14.819929,5.0
1,2016-03-31,Nifty 50,INFY,243457.99,0.818393,0.274475,10.863161,27.0
2,2016-03-31,Nifty 50,KOTAKBANK,69877.08,1.094524,0.274023,14.576403,7.0
3,2016-03-31,Nifty 50,LT,99713.71,1.193147,0.285038,14.685543,6.0
4,2016-03-31,Nifty 50,LUPIN,35321.94,0.734866,0.344207,6.202528,45.0
...,...,...,...,...,...,...,...,...
933,2025-09-30,Nifty 50,HINDALCO,NaN,1.216176,0.277803,15.758734,31.0
934,2025-09-30,Nifty 50,HINDUNILVR,NaN,0.441855,0.195020,11.617741,45.0
935,2025-09-30,Nifty 50,ICICIBANK,NaN,0.880423,0.173273,29.324557,1.0
936,2025-09-30,Nifty 50,COALINDIA,NaN,0.933043,0.213540,20.461686,8.0


In [8]:
# ------------------------------------------------------------
# CONFIGURATION
# ------------------------------------------------------------

TC_COST = 0.0020      # 20 bps transaction cost
MIN_OBS = 200        # minimum observations for estimation

In [26]:
# ------------------------------------------------------------
# PREPARE BENCHMARK RETURNS
# ------------------------------------------------------------
benchmark_returns = index_values.copy()
benchmark_returns["benchmark_return"] = benchmark_returns['Adj Close'].pct_change()
benchmark_returns['Date'] = pd.to_datetime(benchmark_returns['Date'])
benchmark_returns = benchmark_returns[['Date', 'benchmark_return' ]].set_index("Date").dropna()


benchmark_returns

,benchmark_return
Date,
2015-01-05,-0.002031
2015-01-06,-0.029964
2015-01-07,-0.003107
2015-01-08,0.016354
2015-01-09,0.006060
...,...
2025-12-23,0.000181
2025-12-24,-0.001339
2025-12-26,-0.003818


In [27]:
# ------------------------------------------------------------
# PREPARE STOCK RETURN PANEL
# ------------------------------------------------------------

stock_returns = price_data.pivot(
    index="Date",
    columns="Symbol",
    values="daily_return"
)
stock_returns

Symbol,ACC,ADANIENT,ADANIPORTS,AMBUJACEM,APOLLOHOSP,ASIANPAINT,AUROPHARMA,AXISBANK,BAJAJ-AUTO,BAJAJFINSV,...,TECHM,TITAN,TMCV,TRENT,ULTRACEMCO,UPL,VEDL,WIPRO,YESBANK,ZEEL
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,0.015924,0.004898,-0.000626,0.014050,0.001732,0.034572,0.004791,0.022364,-0.000163,-0.008511,...,0.004895,0.007899,NaN,-0.008800,0.029897,0.031177,0.013911,0.008870,0.025435,0.008605
2015-01-05,0.005435,0.016552,0.013935,-0.001299,0.003325,-0.000064,-0.000486,0.006319,0.003650,-0.013287,...,-0.016684,0.008098,NaN,0.001157,0.002042,-0.018595,-0.011246,0.001794,0.002146,-0.002494
2015-01-06,-0.034946,-0.007592,-0.006022,-0.036852,-0.029781,-0.023871,-0.039004,-0.035745,-0.007803,-0.023119,...,-0.010726,-0.034206,NaN,-0.014033,-0.028060,-0.048452,-0.048908,-0.023375,-0.028026,-0.031842
2015-01-07,-0.011637,-0.008858,-0.002330,-0.009228,0.007833,0.020093,0.015858,-0.000802,0.002335,-0.000854,...,-0.004673,0.005098,NaN,0.025949,-0.003951,-0.017936,-0.000239,-0.007611,-0.018532,-0.008290
2015-01-08,0.017514,0.039403,0.041887,0.028623,0.017940,0.063043,0.019592,0.006618,0.001308,0.049855,...,0.016737,-0.004538,NaN,0.009338,0.025338,0.030181,0.007177,0.007207,0.015581,-0.000959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-23,-0.015877,-0.006494,-0.010009,0.012872,0.002764,0.000071,-0.005308,-0.006649,-0.007093,-0.001073,...,-0.008320,-0.002034,0.034977,-0.004136,0.012660,0.005730,0.001281,-0.004658,-0.001837,-0.000653
2025-12-24,-0.008437,-0.011606,0.000469,0.002194,0.013925,-0.007942,-0.005419,0.001061,0.007803,-0.005812,...,-0.000919,-0.004203,-0.040718,0.023918,0.007364,-0.010627,0.020038,-0.012306,-0.002300,-0.000544
2025-12-26,-0.002357,0.003239,-0.004818,0.011950,-0.002231,-0.014001,-0.004293,0.001549,-0.011505,-0.008891,...,-0.011768,0.021155,-0.000245,-0.001002,0.002550,0.001747,0.004681,-0.006566,-0.007838,-0.006535


In [28]:
stock_returns.fillna(method='ffill', inplace=True)

In [29]:
stock_returns.index

DatetimeIndex(['2015-01-02', '2015-01-05', '2015-01-06', '2015-01-07',
               '2015-01-08', '2015-01-09', '2015-01-12', '2015-01-13',
               '2015-01-14', '2015-01-15',
               ...
               '2025-12-16', '2025-12-17', '2025-12-18', '2025-12-19',
               '2025-12-22', '2025-12-23', '2025-12-24', '2025-12-26',
               '2025-12-29', '2025-12-30'],
              dtype='datetime64[ns]', name='Date', length=2714, freq=None)

In [30]:
stock_returns.index.intersection(pd.to_datetime(benchmark_returns.index))

DatetimeIndex(['2015-01-05', '2015-01-06', '2015-01-07', '2015-01-08',
               '2015-01-09', '2015-01-12', '2015-01-13', '2015-01-14',
               '2015-01-15', '2015-01-16',
               ...
               '2025-12-16', '2025-12-17', '2025-12-18', '2025-12-19',
               '2025-12-22', '2025-12-23', '2025-12-24', '2025-12-26',
               '2025-12-29', '2025-12-30'],
              dtype='datetime64[ns]', name='Date', length=2706, freq=None)

In [31]:
common_dates = stock_returns.index.intersection(pd.to_datetime(benchmark_returns.index))
stock_returns = stock_returns.loc[common_dates]
benchmark_returns = benchmark_returns.loc[common_dates]

In [32]:

# ------------------------------------------------------------
# HELPER FUNCTIONS
# ------------------------------------------------------------


def compute_projection_weights(returns_df, benchmark_series):

    df = returns_df.copy()
    df["benchmark"] = benchmark_series

    # Drop rows where benchmark is missing
    df = df.dropna(subset=["benchmark"])

    # Drop stocks with too little history (optional but recommended)
    min_stock_obs = 252  # one year
    valid_stocks = [
        c for c in returns_df.columns
        if returns_df[c].notna().sum() >= min_stock_obs
    ]

    df = df[valid_stocks + ["benchmark"]]

    X = df[valid_stocks].values
    y = df["benchmark"].values

    model = sm.OLS(y, X, missing="drop").fit()
    weights = pd.Series(model.params, index=valid_stocks)

    return weights / weights.sum()



def compute_tracking_error(portfolio_returns, benchmark_series):
    if isinstance(benchmark_series, pd.DataFrame):
        benchmark_series = benchmark_series.iloc[:, 0]

    aligned_benchmark = benchmark_series.reindex(portfolio_returns.index)
    diff = portfolio_returns - aligned_benchmark
    return np.std(diff)


def compute_turnover(prev_weights, new_weights):
    """
    No transaction cost for first portfolio (prev_weights is None)
    """
    if prev_weights is None:
        return 0.0

    aligned = pd.concat([prev_weights, new_weights], axis=1).fillna(0)
    return np.sum(np.abs(aligned.iloc[:, 0] - aligned.iloc[:, 1]))



In [33]:
# ------------------------------------------------------------
# FORWARD SELECTION BY REBALANCE DATE
# ------------------------------------------------------------

# IMPORTANT: prev_rebalance_weights must live OUTSIDE the rebalance loop
prev_rebalance_weights = None

rebalance_dates = sorted(index_narrowing_1["Date"].unique())

for reb_date in rebalance_dates:

    reb_date_dt = pd.to_datetime(reb_date)
    print(f"\nRunning narrowing for rebalance: {reb_date_dt.date()}")

    # ---- Slice ranking for this rebalance
    narrowing_slice = index_narrowing_1[
        index_narrowing_1["Date"] == reb_date
    ]

    ranked_symbols = (
        narrowing_slice
        .sort_values("rank_snr")
        ["Security Symbol"]
        .tolist()
    )

    # ---- Returns available up to rebalance date
    stock_ret_reb = stock_returns.loc[:reb_date]
    bench_ret_reb = benchmark_returns.loc[:reb_date]

    # ---- Initialize for this rebalance
    selected_symbols = []
    prev_net_utility = None
    best_weights = None
    best_k = None

    all_results = []

    for k, symbol in enumerate(ranked_symbols, start=1):

        # Step 2: expand S(k)
        selected_symbols.append(symbol)

        # Align to benchmark calendar (do NOT drop rows across stocks)
        returns_k = stock_ret_reb[selected_symbols]
        returns_k = returns_k.loc[bench_ret_reb.index]

        # Require sufficient benchmark history only
        if len(bench_ret_reb) < MIN_OBS:
            continue

        # Step 3: optimal tracking weights
        weights = compute_projection_weights(returns_k, bench_ret_reb)

        portfolio_returns = returns_k @ weights

        # Step 4: tracking error
        te = compute_tracking_error(portfolio_returns, bench_ret_reb)

        # Step 4: transaction cost
        # TC is evaluated vs PREVIOUS REBALANCE portfolio only
        if prev_rebalance_weights is None:
            tc = 0.0
        else:
            aligned = pd.concat(
                [prev_rebalance_weights, weights],
                axis=1
            ).fillna(0)

            turnover = np.sum(
                np.abs(aligned.iloc[:, 0] - aligned.iloc[:, 1])
            )
            tc = TC_COST * turnover

        # Step 4: net replication utility
        net_replication_utility = -te - tc

        # ---- store portfolio-level diagnostics
        all_results.append({
            "rebalance_date": reb_date,
            "k": k,
            "tracking_error": te,
            "transaction_cost": tc,
            "net_replication_utility": net_replication_utility,
            "weights": json.dumps(weights.to_dict())
        })

        # ---- first feasible portfolio is always accepted
        if best_weights is None:
            best_weights = weights.copy()
            best_k = k
            prev_net_utility = net_replication_utility
            continue

        # ---- marginal stopping rule (THIS matches your paper)
        if net_replication_utility - prev_net_utility <= 0:
            print(f"Stopping at k = {k-1} for {reb_date_dt.date()}")
            break

        prev_net_utility = net_replication_utility
        best_weights = weights.copy()
        best_k = k

    # ---- FINALIZE this rebalance (THIS is when trading occurs)
    if best_weights is not None:
        prev_rebalance_weights = best_weights.copy()
    else:
        print(f"WARNING: no valid portfolio at {reb_date_dt.date()}, carrying forward.")

    # ------------------------------------------------------------
    # OUTPUT
    # ------------------------------------------------------------

    results_df = pd.DataFrame(all_results)

    results_df.to_csv(
        os.path.join(
            base_dir,
            "Rebalance files",
            f"forward_index_narrowing_by_rebalance_{reb_date_dt.date()}.csv"
        ),
        index=False
    )

    print("Final k*:", best_k)
    print(results_df)



Running narrowing for rebalance: 2016-03-31
Stopping at k = 8 for 2016-03-31
Final k*: 8
  rebalance_date  k  tracking_error  transaction_cost  \
0     2016-03-31  1        0.007552               0.0   
1     2016-03-31  2        0.006452               0.0   
2     2016-03-31  3        0.006358               0.0   
3     2016-03-31  4        0.005490               0.0   
4     2016-03-31  5        0.005248               0.0   
5     2016-03-31  6        0.004886               0.0   
6     2016-03-31  7        0.004784               0.0   
7     2016-03-31  8        0.004333               0.0   
8     2016-03-31  9        0.004337               0.0   

   net_replication_utility                                            weights  
0                -0.007552                                  {"HDFCBANK": 1.0}  
1                -0.006452  {"HDFCBANK": 0.6847453909405189, "GRASIM": 0.3...  
2                -0.006358  {"HDFCBANK": 0.5684568821308998, "GRASIM": 0.2...  
3                -0